In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

## 다시 경제 지표 부분을 넣어서 돌린 결과가 가장 높게 나와 최종적으로 이 모델을 선택하였음

In [26]:
f=pd.read_csv('f_with_e2.csv', encoding='euc_kr')

#상승과 하락을 각각 1. 0으로 변환
for i in range(f.shape[0]):
    if f.iloc[i,-1]=='상승':
        f.iloc[i,-1]=1
    else:
        f.iloc[i,-1]=0


In [16]:
f[f['날짜']=='2016-12-29']

,날짜,지수종가,거래대금,상장주식수,시가총액,외국인보유주식수,외국인보유시가총액,신용거래종목수,신용자료일자,회사수,...,금속,oil,미국n년,국고채권(n년),USD,EUR,CNY,JPY,GBP,Y
3961,2016-12-29,2026.46,3009199,41031721,1308440373,6648261,460632479,1282.0,20161227.0,779,...,10995.0,55.0,2.5,3.1,1207.6,1258.44,173.63,1031.7,1476.11,1


In [27]:
X_train=f.iloc[0:3802,1:-1]
X_test=f.iloc[3802:,1:-1]
y_train=f.iloc[0:3802,-1]
y_test=f.iloc[3802:,-1]

In [28]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()

X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

#스케일링 중 발생한 널값에 대해 0(평균)으로 메꿔줌
X_train=np.where(np.isnan(X_train), 0, X_train)
X_test=np.where(np.isnan(X_test), 0, X_test)

In [29]:
# 개별 ML 모델을 위한 Classifier 생성.
knn_clf  = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)

# 최종 Stacking 모델을 위한 Classifier생성. 
lr_final = LogisticRegression(C=10)

In [30]:
# 개별 모델들을 학습. 
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=100, random_state=None)

In [31]:
# 학습된 개별 모델들이 각자 반환하는 예측 데이터 셋을 생성하고 개별 모델의 정확도 측정. 
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도: 0.5443
랜덤 포레스트 정확도: 0.5591
결정 트리 정확도: 0.5401
에이다부스트 정확도: 0.5844 :


In [32]:
pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
print(pred.shape)

# transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦. 
pred = np.transpose(pred)
print(pred.shape)

(4, 474)
(474, 4)


In [33]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

print('최종 메타 모델의 예측 정확도: {0:.4f}'.format(accuracy_score(y_test , final)))

최종 메타 모델의 예측 정확도: 0.6097


C:\Users\chocolet\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [34]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score

def get_clf_eval(y_test , pred):
    confusion = confusion_matrix( y_test, pred)
    accuracy = accuracy_score(y_test , pred)
    precision = precision_score(y_test , pred)
    recall = recall_score(y_test , pred)
    f1 = f1_score(y_test,pred)
    roc_auc = roc_auc_score(y_test, pred)
    print('오차 행렬')
    print(confusion)
    print('정확도: {0:.4f}, 정밀도: {1:.4f}, 재현율: {2:.4f},\
    F1: {3:.4f}, AUC:{4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

In [35]:
get_clf_eval(y_test, final)

오차 행렬
[[ 55 154]
 [ 31 234]]
정확도: 0.6097, 정밀도: 0.6031, 재현율: 0.8830,    F1: 0.7167, AUC:0.5731
